In [1]:
import pandas as pd
import math
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy import stats

from statsmodels.tsa.regime_switching.markov_autoregression import MarkovAutoregression
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression

In [ ]:
#load data
data = pd.read_csv('data/milk.csv')
print data.columns

In [ ]:
reg1 = data[['WW','FMO']].dropna()
reg1 = reg1[reg1['WW']<.4]
reg1 = np.log(reg1)

model = Clusters( reg1['WW'] , np.ones((reg1.shape[0],1)) )
model.weights = np.array([.3,.7])
result = model.fit()
print result.summary()

In [ ]:
#ESC, COOLER ,MILES, GAS, DEL, ESTQTY
reg1 = data[['WW', 'FMO','ESC', 'NUMSCHL','MILES','COOLER', 'GAS','DEL','ESTQTY']].dropna()

#make a little better
reg1['NUMSCHL'] = np.maximum(reg1['NUMSCHL'],1)
reg1['NOSTOP'] = reg1['NUMSCHL']*reg1['DEL']
reg1['MILES'] =reg1['MILES']+1
reg1 = reg1[reg1['WW']<=.3]

#log scale
reg1[['WW', 'FMO', 'MILES', 'GAS','NOSTOP', 'ESTQTY']] = np.log(
                                            reg1[['WW', 'FMO', 'MILES', 'GAS', 'NOSTOP', 'ESTQTY']])

print reg1.mean()



In [ ]:
reg1 = data[['WW','FMO']].dropna()
reg1 = reg1[reg1['WW']<.4]
reg1 = np.log(reg1)
model = sm.OLS(reg1['WW'],sm.add_constant(reg1['FMO']))
result = model.fit()
print result.summary()

In [ ]:
model = MarkovRegression(endog=y, exog=sm.add_constant(x), k_regimes=2, order=0,
                             trend='nc',switching_exog=True, switching_variance=True)
result = model.fit()
print result.summary()